# K-Nearest Neighbor

**Note**: All analysis now onwards has been done on Google colab for fast processing with aid of Graphic
Processing Unit(GPU).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np
import random
random.seed(3)


In [ ]:
df3=pd.read_csv("/content/drive/MyDrive/df3_final.csv")

In [ ]:
df3.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,Pixel_0,Pixel_1,Pixel_2,...,Pixel_3063,Pixel_3064,Pixel_3065,Pixel_3066,Pixel_3067,Pixel_3068,Pixel_3069,Pixel_3070,Pixel_3071,label
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,187,148,190,...,154,132,178,167,150,183,177,153,185,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,25,14,23,...,43,26,42,27,14,24,28,15,25,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,146,133,186,...,143,128,160,139,117,151,116,105,134,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,27,16,31,...,22,16,29,16,12,26,16,12,24,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,134,110,153,...,179,161,202,132,122,158,84,69,101,2


In [ ]:
X = df3.iloc[:,7:3079].values
y = df3.iloc[:, 3079].values


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.20,random_state=30)


In [ ]:
Xtr

array([[123, 109, 167, ..., 124, 123, 184],
       [149, 109, 164, ..., 172, 145, 175],
       [134, 140, 169, ..., 130, 132, 162],
       ...,
       [146, 136, 172, ..., 152, 144, 180],
       [134, 133, 201, ..., 141, 148, 194],
       [183, 172, 246, ..., 179, 177, 244]])

In [ ]:
Xte

array([[158, 163, 192, ..., 159, 162, 185],
       [171, 149, 236, ..., 163, 154, 222],
       [ 16,  16,  26, ...,  66,  58,  80],
       ...,
       [176, 155, 241, ..., 167, 156, 227],
       [203, 182, 231, ..., 198, 185, 230],
       [ 35,  32,  50, ..., 112, 120, 137]])

In [ ]:
ytr

array([4, 2, 5, ..., 2, 5, 5])

In [ ]:
yte

array([5, 5, 2, ..., 5, 5, 3])

In [ ]:
Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3)
Xte_rows = Xte.reshape(Xte.shape[0], 32 * 32 * 3)


In [ ]:
 Xtr_rows

array([[123, 109, 167, ..., 124, 123, 184],
       [149, 109, 164, ..., 172, 145, 175],
       [134, 140, 169, ..., 130, 132, 162],
       ...,
       [146, 136, 172, ..., 152, 144, 180],
       [134, 133, 201, ..., 141, 148, 194],
       [183, 172, 246, ..., 179, 177, 244]])

In [ ]:
class NearestNeighbor(object):
  def __init__(self):
    pass
  def train(self, X, y):
    """ X is N x D where each row is an example. Y is 1-dimension of size N """
     # the nearest neighbor classifier simply remembers all the training data
    self.Xtr = X
    self.ytr = y
  def predict(self, X):
    """ X is N x D where each row is an example we wish to predict label for """
    num_test = X.shape[0]
    # lets make sure that the output type matches the input type
    Ypred = np.zeros(num_test, dtype = self.ytr.dtype)
    # loop over all test rows
    for i in range(num_test):
      # find the nearest training image to the i'th test image
      # using the L1 distance (sum of absolute value differences)
      distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
      min_index = np.argmin(distances) # get the index with smallest distance
      Ypred[i] = self.ytr[min_index] # predict the label of the nearest example
    return Ypred


In [ ]:
nn = NearestNeighbor() # create a Nearest Neighbor classifier class
nn.train(Xtr_rows, ytr) # train the classifier on the training images and labels
Yte_predict = nn.predict(Xte_rows) # predict labels on the test images
# and now print the classification accuracy, which is the average number
# of examples that are correctly predicted (i.e. label matches)
print('accuracy: %f' % ( np.mean(Yte_predict == yte) ))


accuracy: 0.693959


In [ ]:
Accuracy=np.mean(Yte_predict == yte)
Accuracy

0.6939590614078882

In [ ]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
 
results = confusion_matrix(yte, Yte_predict)
print ('Confusion Matrix :')
print(results)
print ('Accuracy Score :',accuracy_score(yte, Yte_predict))
print ('Report : ',classification_report(yte, Yte_predict))


Confusion Matrix :
[[   9    8   20    2    1   20    5]
 [   5   36   16    2    3   41    8]
 [   6    9   74    4   12  100    7]
 [   0    2    5    2    0   13    2]
 [   2   12   27    1   51  130    2]
 [   1   17   47    2   46 1211   15]
 [   0    1    3    0    0   16    7]]
Accuracy Score : 0.6939590614078882
Report :                precision    recall  f1-score   support

           0       0.39      0.14      0.20        65
           1       0.42      0.32      0.37       111
           2       0.39      0.35      0.37       212
           3       0.15      0.08      0.11        24
           4       0.45      0.23      0.30       225
           5       0.79      0.90      0.84      1339
           6       0.15      0.26      0.19        27

    accuracy                           0.69      2003
   macro avg       0.39      0.33      0.34      2003
weighted avg       0.66      0.69      0.67      2003



In [ ]:
class NearestNeighbor1(object):
  def __init__(self):
    pass
  def train(self, X, y):
    """ X is N x D where each row is an example. Y is 1-dimension of size N """
    # the nearest neighbor classifier simply remembers all the training data
    self.Xtr = X
    self.ytr = y
  def predict(self, X):
    """ X is N x D where each row is an example we wish to predict label for """
    num_test = X.shape[0]
    # lets make sure that the output type matches the input type
    Ypred = np.zeros(num_test, dtype = self.ytr.dtype)
    # loop over all test rows
    for i in range(num_test):
      # find the nearest training image to the i'th test image
      # using the L2 distance (sqaured sum of absolute value differences)
      distances = np.sqrt(np.sum(np.square(self.Xtr - X[i,:]), axis =1))
      min_index = np.argmin(distances) # get the index with smallest distance
      Ypred[i] = self.ytr[min_index] # predict the label of the nearest example
    return Ypred


In [ ]:
nn1 = NearestNeighbor1() # create a Nearest Neighbor classifier class
nn1.train(Xtr_rows, ytr) # train the classifier on the training imagesand labels
Yte_predict1 = nn1.predict(Xte_rows) # predict labels on the test images
# and now print the classification accuracy, which is the average number
# of examples that are correctly predicted (i.e. label matches)
print('accuracy: %f' % ( np.mean(Yte_predict1 == yte) ))

accuracy: 0.686970


In [ ]:
Accuracy1=np.mean(Yte_predict1 == yte)
Accuracy1


0.6869695456814778

In [ ]:
results1 = confusion_matrix(yte, Yte_predict1)
print ('Confusion Matrix :')
print(results1)
print ('Accuracy Score :',accuracy_score(yte, Yte_predict1))
print ('Report : ',classification_report(yte, Yte_predict1))

Confusion Matrix :
[[   8   12   20    1    0   18    6]
 [   4   32   18    4    2   44    7]
 [   6    9   85    7   11   91    3]
 [   2    5    5    2    0   10    0]
 [   3    8   34    2   51  125    2]
 [   1   24   72    3   35 1192   12]
 [   0    3    2    0    0   16    6]]
Accuracy Score : 0.6869695456814778
Report :                precision    recall  f1-score   support

           0       0.33      0.12      0.18        65
           1       0.34      0.29      0.31       111
           2       0.36      0.40      0.38       212
           3       0.11      0.08      0.09        24
           4       0.52      0.23      0.31       225
           5       0.80      0.89      0.84      1339
           6       0.17      0.22      0.19        27

    accuracy                           0.69      2003
   macro avg       0.37      0.32      0.33      2003
weighted avg       0.66      0.69      0.66      2003

